In [1]:
import nltk
import numpy as np
import pickle as pkl
import pandas as pd
import copy
import time
import os
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
class_mapping = {}
with open('dbpedia_csv/classes.txt') as txt_file:
    line_num = 1
    for line in txt_file:
        class_mapping[line_num] = line.strip()
        line_num+=1
        
print(class_mapping)

{1: 'Company', 2: 'EducationalInstitution', 3: 'Artist', 4: 'Athlete', 5: 'OfficeHolder', 6: 'MeanOfTransportation', 7: 'Building', 8: 'NaturalPlace', 9: 'Village', 10: 'Animal', 11: 'Plant', 12: 'Album', 13: 'Film', 14: 'WrittenWork'}


In [3]:
train_data = pd.read_csv('dbpedia_csv/train.csv', header = -1, names = ['ClassIndex', 'Title', 'Content'])
test_data = pd.read_csv('dbpedia_csv/test.csv', header = -1, names = ['ClassIndex', 'Title', 'Content'])

In [4]:
training_array = train_data.drop(labels = 'Title', axis = 1).values
test_array = test_data.drop(labels = 'Title', axis = 1).values

In [10]:
ID_count = 0
partition = {}
partition['train'] = []
partition['validation'] = []
labels = {}

start = time.time()
for i in range(len(class_mapping)):
    class_index = i+1
    train_count = 0
    num_sent = 0
    sample_vec = []
    for j in range(len(training_array)):
        if training_array[j,0] == class_index:
            par_vec = []
            for cur_word in word_tokenize(training_array[j,1]):
                par_vec.append(cur_word)
                sample_vec.append(copy.copy(par_vec))
                if (train_count+1)%32 == 0:
                    pkl.dump((class_index, sample_vec), open('Training/'+str(ID_count), 'wb'))
                    labels[str(ID_count)] = class_index
                    sample_vec = []
                    if num_sent < 35000:
                        partition['train'].append(str(ID_count))
                    else:
                        partition['validation'].append(str(ID_count))
                    ID_count += 1
                train_count += 1
            num_sent += 1
    print('Elapsed time: {}'.format((time.time()-start)/60))
pkl.dump((partition, labels),open('Training/Metadata', 'wb'))

Elapsed time: 0.40436224540074667
Elapsed time: 0.821736752986908
Elapsed time: 1.215203829606374
Elapsed time: 1.5890655517578125
Elapsed time: 2.0093705296516418
Elapsed time: 2.4438748598098754
Elapsed time: 2.934901773929596
Elapsed time: 3.290975852807363
Elapsed time: 3.6402346452077228
Elapsed time: 3.965379869937897
Elapsed time: 4.363205095132192
Elapsed time: 4.777144082387289
Elapsed time: 5.185762095451355
Elapsed time: 5.63558255036672


In [14]:
start = time.time()
for i in range(len(class_mapping)):
    class_index = i+1
    test_count = 0
    for j in range(len(test_array)):
        if test_array[j,0] == class_index:
            par_vec = []
            for cur_word in word_tokenize(test_array[j,1]):
                par_vec.append(cur_word)
                sample_len = len(par_vec)
                if not os.path.exists('Testing/Class{}/{}'.format(class_index,sample_len)):
                    os.makedirs('Testing/Class{}/{}'.format(class_index,sample_len))
                pkl.dump((class_index, par_vec), open('Testing/Class{}/{}/{}'.format(class_index, sample_len, test_count),'wb'))
                test_count += 1
    print('Elapsed time: {}'.format((time.time()-start)/60))

Elapsed time: 0.26567095120747886
Elapsed time: 0.5692348043123882
Elapsed time: 0.8823874950408935
Elapsed time: 1.1735702276229858
Elapsed time: 1.4765604694684347
Elapsed time: 1.7998019615809122
Elapsed time: 2.1730777740478517
Elapsed time: 2.4402572830518086
Elapsed time: 2.6664554437001544
Elapsed time: 2.8730860074361164
Elapsed time: 3.1306859930356343
Elapsed time: 3.418167742093404
Elapsed time: 3.7059060454368593
Elapsed time: 3.9970598816871643
